In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import yaml
import os
from openpyxl import load_workbook

In [ ]:
flag_project = 1 # 0--regular; 1--repartrition

In [ ]:
if flag_project == 0:
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()    

In [ ]:
key_subworks = info_basic['key_subworks']
key_subworks

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
stalist_all = stainfo['Station'].tolist() 
lat_all =  stainfo['latitude'].tolist() 
lon_all =  stainfo['longitude'].tolist() 
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [ ]:
stalistname = dir_project+ info_basic['stalistname']
stainfo_all = pd.read_excel(stalistname,sheet_name=None)

In [ ]:
def calculate_centroid(coordinates):
    return np.mean(coordinates, axis=0)

In [ ]:
def loc_key(key_subwork):
    global stalist_all
    global lat_all
    global lon_all
    key = 'R'+key_subwork[0:2]+key_subwork[3:]
    if key in stalist_all:
        lat_this = lat_all[stalist_all.index(key)]
        lon_this = lon_all[stalist_all.index(key)]
    else:
        key1 = 'R' + str(int(key_subwork[0:2])-1).zfill(2)+key_subwork[3:5]
        key2 = 'R' + str(int(key_subwork[0:2])+1).zfill(2)+key_subwork[3:5]
        indx1 = stalist_all.index(key1)
        indx2 = stalist_all.index(key2)
        lat_this= (lat_all[indx1]+lat_all[indx2])/2
        lon_this = (lon_all[indx1]+lon_all[indx2])/2
    return lon_this,lat_this

In [ ]:
lon_centroid_all = {}
lat_centroid_all = {}

In [ ]:
for key_subwork in key_subworks:
    stainfo = stainfo_all[key_subwork]
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist()
    lon = stainfo['longitude'].tolist()
    lon_centroid,lat_centroid = calculate_centroid(np.array([lon,lat]).T)
    lon_centroid_all[key_subwork] = lon_centroid
    lat_centroid_all[key_subwork] = lat_centroid

### Write into xlsx file

In [ ]:
"""
books = load_workbook(stalistname)
writers = pd.ExcelWriter(stalistname, engine='openpyxl')
writers.book = books
df=pd.DataFrame()
df['key_subwork'] = list(key_subworks)
df['lon_centroid'] = list(lon_centroid_all.values())
df['lat_centroid'] = list(lat_centroid_all.values())
df.to_excel(writers,sheet_name='location',index=False)
writers.save()
"""

In [ ]:
filename = dir_project + 'subwork_location.xlsx'
df=pd.DataFrame()
df['key_subwork'] = [str(key_subwork) for key_subwork in key_subworks]
df['lon_centroid'] = list(lon_centroid_all.values())
df['lat_centroid'] = list(lat_centroid_all.values())
df.to_excel(filename,sheet_name='location',index=False)